### 기본 개념: Nodes, Edges 와 State

In [ ]:
# git : git clone https://github.com/moonjukhim/langgraph.git

%pip install -r requirements.txt

In [ ]:
from langgraph.graph import END, START, StateGraph
from typing import TypedDict

In [ ]:
class InputState(TypedDict):
    string_value: str
    numeric_value: int


def modify_state(input: InputState):
    print(f"Current value: {input}")
    return input


In [ ]:
graph = StateGraph(InputState)

graph.add_node("branch_a", modify_state)
graph.add_node("branch_b", modify_state)
graph.add_edge(START, "branch_a")
graph.add_edge("branch_a", "branch_b")
graph.add_edge("branch_b", END)

graph.set_entry_point("branch_a")

mygraph = graph.compile()

### Visualize graph

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        mygraph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
mygraph.invoke({"string_value": "a"})

In [ ]:
mygraph.invoke({"string_value": "a", "numeric_value": 1})

In [ ]:
def modify_state(input: InputState):
    input["string_value"] += "a"
    input["numeric_value"] += 1
    return input


graph = StateGraph(InputState)

graph.add_node("branch_a", modify_state)
graph.add_node("branch_b", modify_state)
graph.add_edge("branch_a", "branch_b")
graph.add_edge("branch_b", END)


graph.set_entry_point("branch_a")

mygraph = graph.compile()

In [ ]:
# try:
#    mygraph.invoke({"string_value": "a"})
# except KeyError as e:
#    print(e)

In [ ]:
mygraph.invoke({"string_value": "a", "numeric_value": 1})

### LangChain(LCEL) Vs. LangGraph

In [ ]:
from langchain_core.runnables import RunnableLambda

runnable = RunnableLambda(modify_state)
chain = runnable | runnable

chain.invoke({"string_value": "a", "numeric_value": 1})

### Cycles and Conditional Edges

In [ ]:
def modify_state(input: InputState):
    input["string_value"] += "a"
    input["numeric_value"] += 1
    return input


def router(input: InputState):
    if input["numeric_value"] < 5:
        return "branch_a"
    else:
        return "__end__"


graph = StateGraph(InputState)

graph.add_node("branch_a", modify_state)
graph.add_node("branch_b", modify_state)
graph.add_edge("branch_a", "branch_b")
graph.add_conditional_edges(
    "branch_b", router, {"branch_a": "branch_a", "__end__": END}
)

graph.set_entry_point("branch_a")

mygraph = graph.compile() 


In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        mygraph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
mygraph.invoke({"string_value": "a", "numeric_value": 1})